In [1]:
# set working directory :
import os
pwd = os.getcwd() + "/../"
os.chdir(pwd)

In [2]:
from quanta.clients.yfinance import YahooFinanceClient
from datetime import datetime, timedelta

n=365
from_date = datetime.now() - timedelta(days=n)
to_date = datetime.now() - timedelta(days=1)

yh = YahooFinanceClient()
df_init = yh.get_price(
    "^SPMCBRT",
    from_date=from_date.strftime("%Y-%m-%d"),
    to_date=to_date.strftime("%Y-%m-%d"),
    interval="1h", 
    postclean=True
)

df_init.to_pandas()

,timestamp,datetime,open,high,low,close,volume
0,1731335400,2024-11-11 15:30:00,1230.488281,1234.161621,1225.380615,1225.805054,0
1,1731339000,2024-11-11 16:30:00,1225.700684,1228.165527,1222.813110,1227.148804,0
2,1731342600,2024-11-11 17:30:00,1226.339233,1228.886230,1223.311157,1228.270020,0
3,1731346200,2024-11-11 18:30:00,1228.338379,1228.338379,1224.627441,1225.645874,0
4,1731349800,2024-11-11 19:30:00,1225.509033,1230.325684,1224.981812,1226.921143,0
...,...,...,...,...,...,...,...
1486,1762529400,2025-11-07 16:30:00,1231.090942,1233.051147,1229.179321,1230.926147,0
1487,1762533000,2025-11-07 17:30:00,1230.401978,1235.671265,1229.198853,1230.858154,0
1488,1762536600,2025-11-07 18:30:00,1230.751343,1232.595093,1224.793091,1232.595093,0
1489,1762540200,2025-11-07 19:30:00,1232.236084,1234.070190,1231.052246,1232.362305,0


In [3]:
# import strategy :
from quanta.ta_clients.optimization_strategy import StrategyClient
strat = StrategyClient()

/home/cmileto/miniconda3/envs/ta/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
INDICATOR_CLASSES = strat.get_seed_indicator_config()
INDICATOR_CLASSES


{'SMA': quanta.utils.ta.SMA,
 'RSI': quanta.utils.ta.RSI,
 'MACD': quanta.utils.ta.MACD,
 'BollingerBands': quanta.utils.ta.BollingerBands,
 'ATR': quanta.utils.ta.ATR}

In [5]:
OPTIMIZATION_CONFIG = strat.get_seed_optimization_config()
OPTIMIZATION_CONFIG

{'strategy_name': 'simple_strategy',
 'indicators': {'SMA_short': {'class': 'SMA',
   'params': {'period': {'type': 'int', 'low': 5, 'high': 50}}},
  'SMA_long': {'class': 'SMA',
   'params': {'period': {'type': 'int', 'low': 20, 'high': 200}}},
  'RSI': {'class': 'RSI',
   'params': {'period': {'type': 'int', 'low': 14, 'high': 21}}},
  'ATR': {'class': 'ATR',
   'params': {'period': {'type': 'int', 'low': 10, 'high': 30}}},
  'BollingerBands': {'class': 'BollingerBands',
   'params': {'period': {'type': 'int', 'low': 15, 'high': 30},
    'std_dev': {'type': 'float', 'low': 2.0, 'high': 3.0}}},
  'MACD': {'class': 'MACD',
   'params': {'fast_period': {'type': 'int', 'low': 8, 'high': 16},
    'slow_period': {'type': 'int', 'low': 20, 'high': 30},
    'signal_period': {'type': 'int', 'low': 7, 'high': 12}}}},
 'strategy': {'stop_loss': {'type': 'float',
   'low': 0.005,
   'high': 0.02,
   'log': True},
  'take_profit': {'type': 'float', 'low': 0.01, 'high': 0.05, 'log': True},
  'posi

In [6]:
strat.print_strategies()

Available strategies:

  simple_strategy:
    Description: Simple strategy based on SMA crossovers and RSI levels.
    Required indicators: ['SMA_short', 'SMA_long', 'RSI', 'MACD', 'ATR', 'BollingerBands']


In [10]:
STRATEGY = strat.get_strategy_fct("simple_strategy")

In [11]:
# test module OptimizationClient : 
from quanta.ta_clients.optimization import OptimizationClient
opt = OptimizationClient(optimization_config=OPTIMIZATION_CONFIG)


✓ Strategy 'simple_strategy' initialized with indicators: ['SMA_short', 'SMA_long', 'RSI', 'MACD', 'ATR', 'BollingerBands']


In [12]:
opt.optimize(
    df = df_init, 
    backtest_func=STRATEGY, 
    verbose=False)

[I 2025-11-10 11:43:18,842] A new study created in memory with name: trading_strategy_optimization
Best trial: 839. Best value: 30.1094: 100%|██████████| 1000/1000 [00:24<00:00, 40.63it/s, 24.61/3600 seconds]


In [13]:
opt.analyze_parameter_importance()
useless_params = opt.get_useless_parameters(threshold=0.01)
opt.plot_importance()  # Graphique interactif


PARAMETER IMPORTANCE (FANOVA)
ℹ️  Number of omitted trials: 55
stop_loss                     : 0.3650 ✓ Useful
SMA_short_period              : 0.1375 ✓ Useful
take_profit                   : 0.1230 ✓ Useful
ATR_period                    : 0.1030 ✓ Useful
position_size                 : 0.0734 ✓ Useful
macd_hist_sell_threshold      : 0.0713 ✓ Useful
RSI_period                    : 0.0385 ✓ Useful
macd_hist_buy_threshold       : 0.0352 ✓ Useful
MACD_signal_period            : 0.0265 ✓ Useful
BollingerBands_std_dev        : 0.0149 ✓ Useful
MACD_fast_period              : 0.0060 ⚠️  USELESS
MACD_slow_period              : 0.0056 ⚠️  USELESS

LOW IMPACT PARAMETERS ANALYSIS

⚠️  Technical indicators with low impact (< 0.01):
  - MACD_fast_period               (importance: 0.0091)

✓ All strategy parameters have significant impact



ℹ️  Number of omitted trials: 55


In [16]:
from quanta.utils.ta import TAClient

indicators = opt.get_best_indicators()
indicators


In [17]:

ta_client = TAClient()
df = ta_client.calculate_indicators(df_init, indicators)

from quanta.utils.trace import Candlesticks, Volume, Line

traces = [
    Candlesticks(),
    Line('hl_avg', name='HL Average', color='purple'),
    Volume()
]


from quanta.clients.chart import ChartClient

chart_client = ChartClient()
chart_client.plot(df, "AAPL", traces=traces, indicators=indicators, theme='professional')

Plotting 1491 bars for AAPL with x_axis_type='row_nb'


In [18]:
trades = opt.get_trades_history(df=df_init)
trades

  Signals generated: 377
  📊 Total signals: 377 (Long: 166, Short: 211)
  💼 Executed trades: 377
  📅 Period: 2024-12-13 19:30:00 → 2025-11-07 20:30:00 (329 days = 0.90 years)
  📊 Trades: 377 | Return: 0.3106 | Sharpe: 30.11 | DD: 3.18%


timestamp,position_number,action,price,quantity_usd,position_size,pnl,cumulative_capital
datetime[μs],i64,str,f64,f64,f64,f64,f64
2024-12-16 15:30:00,0,"""SELL""",1274.555176,0.775127,0.774105,0.00132,1.00132
2024-12-16 16:30:00,1,"""SELL""",1265.893066,0.779205,0.774105,0.005261,1.006588
2024-12-16 18:30:00,2,"""SELL""",1269.523926,0.779812,0.774105,0.000779,1.007372
2024-12-16 19:30:00,3,"""SELL""",1268.996582,0.780063,0.774105,0.000322,1.007696
2024-12-16 20:30:00,4,"""SELL""",1266.87854,0.781071,0.774105,0.001292,1.008998
…,…,…,…,…,…,…,…
2025-11-03 18:30:00,372,"""SELL""",1257.997192,1.044547,0.774105,-0.000072,1.349361
2025-11-03 19:30:00,373,"""SELL""",1253.486816,1.047446,0.774105,0.002775,1.353106
2025-11-03 20:30:00,374,"""SELL""",1253.292847,1.047572,0.774105,0.00012,1.353268


In [19]:
chart_client = ChartClient()
chart_client.plot(
    df, "AAPL", 
    traces=traces, 
    indicators=indicators, 
    trades_df=trades, 
    theme='professional'
)

Plotting 1491 bars for AAPL with x_axis_type='row_nb'
With 377 trades
Debug: 377 trades after processing
shape: (5, 4)
┌─────────────────────┬─────────┬────────┬─────────────┐
│ timestamp           ┆ x_value ┆ action ┆ price       │
│ ---                 ┆ ---     ┆ ---    ┆ ---         │
│ datetime[μs]        ┆ i64     ┆ str    ┆ f64         │
╞═════════════════════╪═════════╪════════╪═════════════╡
│ 2024-12-16 15:30:00 ┆ 142     ┆ SELL   ┆ 1274.555176 │
│ 2024-12-16 16:30:00 ┆ 143     ┆ SELL   ┆ 1265.893066 │
│ 2024-12-16 18:30:00 ┆ 145     ┆ SELL   ┆ 1269.523926 │
│ 2024-12-16 19:30:00 ┆ 146     ┆ SELL   ┆ 1268.996582 │
│ 2024-12-16 20:30:00 ┆ 147     ┆ SELL   ┆ 1266.87854  │
└─────────────────────┴─────────┴────────┴─────────────┘
